In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
with open("config/db_credentials.json", "r") as f:
    db_credentials = json.load(f)

In [3]:
def create_engine_connection(db_credentials: dict):
    return create_engine(
        f"postgresql+psycopg2://{db_credentials['user']}:{db_credentials['password']}@{db_credentials['host']}:{db_credentials['port']}/{db_credentials['dbname']}"
    )

In [4]:
query = "SELECT * FROM public.cars_scraped"
engine = create_engine_connection(db_credentials)

In [5]:
df = pd.read_sql(query, engine)

In [6]:
df

,id,created_at,manufacturer,model,version,month,year,kms,fuel,transmission,power_hp,no_doors,color,seller,price_cash,price_financed,link
0,1,2025-02-11 23:50:19.307619,audi,a4,AVANT ADVANCED EDITION 2.0 TDI 190 CV 5P,5,2018,133382,d,a,190,5,gris,prof,17200.0,248.24,https://www.coches.com/coches-segunda-mano/oca...
1,2,2025-02-11 23:50:19.307619,audi,a4,s-line,11,2022,47800,hg,a,136,5,gris,prof,27128.0,25772.00,https://www.coches.com/coches-segunda-mano/oca...
2,3,2025-02-11 23:50:19.307619,audi,a4,S line 35 TDI 120kW (163CV) S tronic,3,2024,18120,hd,a,163,4,blanco,prof,35900.0,34900.00,https://www.coches.com/coches-segunda-mano/oca...
3,4,2025-02-11 23:50:19.307619,audi,a4,2.0 TDI 143cv DPF,1,2011,211158,d,a,143,4,negro,prof,9800.0,NaN,https://www.coches.com/coches-segunda-mano/oca...
4,5,2025-02-11 23:50:19.307619,audi,a4,s-line,8,2019,106014,hg,a,150,5,negro,prof,19289.0,18324.00,https://www.coches.com/coches-segunda-mano/oca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233061,233081,2025-03-02 00:56:46.320460,opel,kaddet comercial,Combi 9 1.6CDTi Biturbo S-S 29 L2 125,5,2018,87000,d,m,125,4,blanco,prof,21538.0,NaN,https://www.coches.com/coches-segunda-mano/oca...
233062,233082,2025-03-02 00:56:46.320460,opel,kaddet comercial,"Cargo L1H1 2,2t",11,2018,119005,d,m,95,3,blanco,prof,9010.0,8290.00,https://www.coches.com/coches-segunda-mano/oca...
233063,233083,2025-03-02 00:56:46.320460,opel,kaddet comercial,Tour 1.3CDTI Expression L1H1 95,8,2017,56330,d,m,95,5,blanco,prof,12500.0,NaN,https://www.coches.com/coches-segunda-mano/oca...
233064,233084,2025-03-02 00:56:46.320460,opel,kaddet comercial,CARGO L 650KG DIESEL 1.5 100HPS&S MT E6,3,2023,40064,d,m,100,5,blanco,prof,16111.0,262.62,https://www.coches.com/coches-segunda-mano/oca...


# Feature engineering

## Age

In [7]:
current_year = datetime.now().year + datetime.now().month / 12
current_year

2025.25

In [8]:
col = "age"
df[col] = np.maximum(0, current_year - (df["year"] + df["month"] / 12))

df[["manufacturer", "model", "year", "month", col]].sample(10)

,manufacturer,model,year,month,age
169396,bmw,x5,2020,8,4.583333
104037,seat,arona,2021,5,3.833333
41076,ford,mondeo,2020,10,4.416667
46595,hyundai,i20,2021,3,4.000000
143,audi,a4,2023,1,2.166667
115485,toyota,c hr,2021,6,3.750000
29618,fiat,500,2022,11,2.333333
68286,nissan,qashqai,2021,10,3.416667
9906,bmw,serie 3,2008,2,17.083333
219297,mercedes,clase v,2019,10,5.416667


## Age bins

In [9]:
col = "age_bins"
df[col] = pd.cut(df["age"], bins=range(0, 61, 3), labels=[f"[{i}-{i+3})" for i in range(0, 60, 3)], right=False)
df[["manufacturer", "model", "age", col]].sample(10)

,manufacturer,model,age,age_bins
161792,audi,q5,9.583333,[9-12)
27004,cupra,formentor,2.666667,[0-3)
106977,seat,ibiza,4.166667,[3-6)
94582,peugeot,rifter n1,4.333333,[3-6)
162669,bmw,serie 1,4.083333,[3-6)
12327,bmw,x1,4.250000,[3-6)
186947,fiat,500,3.500000,[3-6)
145276,kia,xceed,2.333333,[0-3)
192968,ford,focus,0.166667,[0-3)
154051,ssangyong,tivoli,1.750000,[0-3)


## Kms per year

In [10]:
col = "kms_per_year"
df[col] = df["kms"] / (df["age"] + 1/12)
df[["manufacturer", "model", "kms", "age", col]].sample(10)

,manufacturer,model,kms,age,kms_per_year
53754,mercedes,clase a,24959,1.583333,14975.400000
90374,peugeot,508,125238,5.833333,21166.985915
63904,mercedes,clase v,164127,3.916667,41031.750000
5936,audi,q3,93268,4.250000,21523.384615
125452,volkswagen,t cross,58217,2.750000,20547.176471
55770,mercedes,clase a,99000,4.833333,20135.593220
211307,mercedes,clase a,10299,1.250000,7724.250000
152106,mini,mini,3000,1.166667,2400.000000
183075,cupra,formentor,88316,2.166667,39251.555556
205066,hyundai,tucson,32818,2.750000,11582.823529


## Kms per year bins

In [11]:
col = "kms_per_year_bins"
df[col] = pd.cut(df["kms_per_year"], bins=[0, 10000, 20000, 10000000], labels=["low", "medium", "high"], right=False)
df[["manufacturer", "model", "kms", "age", "kms_per_year", col]].sample(10)

,manufacturer,model,kms,age,kms_per_year,kms_per_year_bins
45224,ford,transit custom m1 v710,127617,5.166667,24308.000000,high
47418,hyundai,i30,8,0.333333,19.200000,low
107595,seat,ibiza,85936,3.833333,21941.106383,high
190279,fiat,talento,77500,5.833333,13098.591549,medium
64466,mercedes,sprinter,345000,10.500000,32598.425197,high
123736,volkswagen,passat,50528,5.166667,9624.380952,low
188855,fiat,doblo,15900,2.000000,7632.000000,low
119026,volkswagen,arteon,78367,6.333333,12213.038961,medium
6264,audi,q3,27762,2.083333,12813.230769,medium
129779,volvo,s60,190000,19.500000,9702.127660,low


In [12]:
df[df["kms_per_year_bins"].isnull()]

,id,created_at,manufacturer,model,version,month,year,kms,fuel,transmission,...,no_doors,color,seller,price_cash,price_financed,link,age,age_bins,kms_per_year,kms_per_year_bins


## Weight

## Power to weight ratio

## Brand new car price

## Average price of each model

In [13]:
col = "avg_model_price"
df[col] = df.groupby("model")["price_cash"].transform("mean")
df[["manufacturer", "model", "price_cash", col]].sample(10)

,manufacturer,model,price_cash,avg_model_price
11426,bmw,serie 5,45900.0,31210.452555
74010,opel,grandland,31850.0,24558.286445
29592,fiat,500,15950.0,11133.275799
165636,bmw,serie 3,16299.0,26668.399599
53770,mercedes,clase a,20850.0,29264.936669
66843,nissan,micra,12890.0,12575.680172
140892,kia,niro,19795.0,26605.631841
62650,mercedes,clase glc,43985.0,41812.760380
100790,renault,kangoo,10300.0,10276.764770
139115,jeep,renegade,20890.0,19111.810219


## Price deviation from average

In [14]:
# col = "std_model_price"
# df[col] = df.groupby("model")["price_cash"].transform("std")
# df[["manufacturer", "model", "price_cash", "avg_model_price", col]].sample(10)

## Car Segment

In [15]:
with open("/home/ubuntu/car_price_checker_2/config/models_vs_segment.json", "r") as f:
    models_vs_segment = json.load(f)

In [16]:
col = "model_segment"
df[col] = df["model"].map(models_vs_segment)
df[col] = df[col].replace("Unknown", None)
df[col] = df[col].str.lower()
df[["manufacturer", "model", col]].sample(10)

,manufacturer,model,model_segment
128368,volkswagen,tiguan,c
51939,mazda,cx 5,j
186601,fiat,500,a
113014,skoda,kodiaq,j
158895,audi,a6,e
93139,peugeot,rifter,m
129942,volvo,v60 cross country,d
219320,mercedes,clase v,m
115358,toyota,c hr,j
216655,mercedes,clase gla,j


## Brand popularity

In [17]:
col = "brand_rel_freq"
df[col] = df["manufacturer"].map(df["manufacturer"].value_counts(normalize=True))
df[["manufacturer", col]].sample(10)

,manufacturer,brand_rel_freq
42574,ford,0.084684
161321,audi,0.064398
153715,smart,0.000957
40554,ford,0.084684
150752,mg,0.009058
175060,citroen,0.104957
84371,peugeot,0.079209
208823,mercedes,0.107038
200520,ford,0.084684
225610,nissan,0.042318


In [18]:
df["manufacturer"].value_counts(normalize=True)

manufacturer
mercedes    0.107038
citroen     0.104957
ford        0.084684
peugeot     0.079209
audi        0.064398
              ...   
alpine      0.000017
aiways      0.000013
daewoo      0.000013
hummer      0.000004
galloper    0.000004
Name: proportion, Length: 61, dtype: float64

## Brand country

In [19]:
with open("config/manufacturers_details.json", "r") as f:
    manufacturers_details = json.load(f)

In [20]:
col = "brand_country"
df[col] = df["manufacturer"].map(lambda x: manufacturers_details.get(x, {}).get("country", "unknown"))
df[col] = df[col].str.lower()
df[["manufacturer", col]].sample(10)

,manufacturer,brand_country
150181,mg,china
207272,mazda,japan
227310,opel,germany
229763,opel,germany
156305,audi,germany
145643,kia,south korea
33483,fiat,italy
136835,infiniti,japan
7037,audi,germany
16721,citroen,france


## Exclusivity level

In [21]:
col = "brand_exclusivity"
df[col] = df["manufacturer"].map(lambda x: manufacturers_details.get(x, {}).get("exclusivity_level", "unknown"))
df[col] = df[col].str.lower()
df[["manufacturer", col]].sample(10)

,manufacturer,brand_exclusivity
5280,audi,premium
131997,volvo,premium
33217,fiat,mainstream
212867,mercedes,premium
25443,citroen,mainstream
205982,hyundai,mainstream
147493,land rover,luxury
25041,citroen,mainstream
196301,ford,mainstream
108638,seat,mainstream


## Group

In [22]:
col = "brand_group"
df[col] = df["manufacturer"].map(lambda x: manufacturers_details.get(x, {}).get("group", "unknown"))
df[col] = df[col].str.lower().str.replace(" ", "_").str.replace("-", "_")
df[["manufacturer", col]].sample(10)

,manufacturer,brand_group
56855,mercedes,mercedes_benz_group
136558,ds,stellantis
11513,bmw,bmw_group
144978,kia,hyundai_motor_group
178258,citroen,stellantis
179674,citroen,stellantis
118594,toyota,toyota_motor_corporation
212573,mercedes,mercedes_benz_group
162112,audi,volkswagen_group
65024,mercedes,mercedes_benz_group


# Model popularity

In [23]:
col = "model_rel_freq"
df[col] = df["model"].map(df["model"].value_counts(normalize=True))
df[["model", col]].sample(10)

,model,model_rel_freq
19069,c3 aircross,0.010053
201260,i10,0.002969
93318,rifter,0.005492
219993,sprinter,0.002999
151135,countryman,0.001969
210763,clase a,0.020664
227277,astra,0.007337
106428,ateca,0.006427
199825,transit,0.002832
70039,astra,0.007337


In [24]:
df["model"].value_counts(normalize=True)

model
500               0.025903
qashqai           0.022522
clase a           0.020664
tucson            0.018677
c4 spacetourer    0.018046
                    ...   
splash            0.000004
e tron gt         0.000004
kizashi           0.000004
370z              0.000004
kadett            0.000004
Name: proportion, Length: 709, dtype: float64

# Imputation of missing records

In [25]:
col = "model_segment"
df[col] = df[col].fillna("unknown")

In [26]:
df[col].isna().sum()

np.int64(0)

# Export df

In [27]:
df

,id,created_at,manufacturer,model,version,month,year,kms,fuel,transmission,...,age_bins,kms_per_year,kms_per_year_bins,avg_model_price,model_segment,brand_rel_freq,brand_country,brand_exclusivity,brand_group,model_rel_freq
0,1,2025-02-11 23:50:19.307619,audi,a4,AVANT ADVANCED EDITION 2.0 TDI 190 CV 5P,5,2018,133382,d,a,...,[6-9),19284.144578,medium,23708.674474,d,0.064398,germany,premium,volkswagen_group,0.007144
1,2,2025-02-11 23:50:19.307619,audi,a4,s-line,11,2022,47800,hg,a,...,[0-3),19779.310345,medium,23708.674474,d,0.064398,germany,premium,volkswagen_group,0.007144
2,3,2025-02-11 23:50:19.307619,audi,a4,S line 35 TDI 120kW (163CV) S tronic,3,2024,18120,hd,a,...,[0-3),16726.153846,medium,23708.674474,d,0.064398,germany,premium,volkswagen_group,0.007144
3,4,2025-02-11 23:50:19.307619,audi,a4,2.0 TDI 143cv DPF,1,2011,211158,d,a,...,[12-15),14818.105263,medium,23708.674474,d,0.064398,germany,premium,volkswagen_group,0.007144
4,5,2025-02-11 23:50:19.307619,audi,a4,s-line,8,2019,106014,hg,a,...,[3-6),18708.352941,medium,23708.674474,d,0.064398,germany,premium,volkswagen_group,0.007144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233061,233081,2025-03-02 00:56:46.320460,opel,kaddet comercial,Combi 9 1.6CDTi Biturbo S-S 29 L2 125,5,2018,87000,d,m,...,[6-9),12578.313253,medium,17114.382166,unknown,0.058331,germany,mainstream,stellantis,0.002695
233062,233082,2025-03-02 00:56:46.320460,opel,kaddet comercial,"Cargo L1H1 2,2t",11,2018,119005,d,m,...,[6-9),18546.233766,medium,17114.382166,unknown,0.058331,germany,mainstream,stellantis,0.002695
233063,233083,2025-03-02 00:56:46.320460,opel,kaddet comercial,Tour 1.3CDTI Expression L1H1 95,8,2017,56330,d,m,...,[6-9),7347.391304,low,17114.382166,unknown,0.058331,germany,mainstream,stellantis,0.002695
233064,233084,2025-03-02 00:56:46.320460,opel,kaddet comercial,CARGO L 650KG DIESEL 1.5 100HPS&S MT E6,3,2023,40064,d,m,...,[0-3),19230.720000,medium,17114.382166,unknown,0.058331,germany,mainstream,stellantis,0.002695


In [28]:
df.to_parquet("/home/ubuntu/car_price_checker_2/data/feature_engineering/df.parquet")

In [29]:
df.isnull().sum()

id                       0
created_at               0
manufacturer             0
model                    0
version                  0
month                    0
year                     0
kms                      0
fuel                     0
transmission             0
power_hp                 0
no_doors                 0
color                    0
seller                   0
price_cash               0
price_financed       76202
link                     0
age                      0
age_bins                 0
kms_per_year             0
kms_per_year_bins        0
avg_model_price          0
model_segment            0
brand_rel_freq           0
brand_country            0
brand_exclusivity        0
brand_group              0
model_rel_freq           0
dtype: int64